In [21]:
from pathlib import Path
import json
from tqdm import tqdm
import random
import anthropic

In [25]:
data = Path("data") / "evaluation" / "single_turn"

In [26]:
paths = [p for p in data.glob("*.json") if "fake" not in str(p) and "syn" not in str(p)]
all_prompts = {p.stem: json.loads(p.read_text()) for p in paths}
{k:len(v) for k,v in all_prompts.items()}

{'building_location': 1128,
 'wheelchair_accessible': 1128,
 'open_now': 1211,
 'closed_until': 1211,
 'open_until': 1211,
 'building_type': 828,
 'navigation_link': 2256,
 'opening_hours': 1128,
 'building_category': 828,
 'open_website': 2256}

# Plan

Umformulieren von "prompts"
Eingießen in Pydantic

In [27]:
process = {}

for prompt_type in all_prompts.keys():
    if prompt_type in ("building_type", "building_category"):
        continue
    
    pos = []
    neg = []
    for prompt in all_prompts[prompt_type]:
        if "leider" in str(prompt) or "keine" in str(prompt):
            neg.append(prompt)
        else:
            pos.append(prompt)
        
    # sample 50 positives / 50 negatives per type
    k_pos = min(50, len(pos))
    k_neg = min(50, len(neg))
    
    sampled_pos = random.sample(pos, k_pos)
    sampled_neg = random.sample(neg, k_neg)

    print(prompt_type)
    print(len(sampled_pos), len(sampled_neg))
    process[prompt_type] = sampled_pos + sampled_neg
    

building_location
50 50
wheelchair_accessible
50 50
open_now
50 50
closed_until
50 50
open_until
50 50
navigation_link
50 0
opening_hours
50 50
open_website
50 50


In [33]:
API_KEY = "<you wish, NSA :)>"

In [32]:
client = anthropic.Anthropic(api_key=API_KEY)
batch_size = 25

system_prompt = (
    "You will receive a list of input prompts. For each, rewrite it in a different way "
    "that retains the original meaning and intent, but uses different wording or phrasing. "
    "Your output must be a single string, with all reformulated prompts in the same order, "
    "separated ONLY by a vertical bar '|'. Do not include any extra text, formatting, or explanations. "
)

for prompt_type, prompts in process.items():
    print(prompt_type)
    if prompt_type in ("building_location", "wheelchair_accessible", "open_now", "closed_until"):
        print("skipping", prompt_type)
        continue
    
    for start in tqdm(range(0, len(prompts), batch_size)):
        batch = prompts[start:start + batch_size]

        # Skip batch if already processed
        if "reformulated_prompt" in batch[0]["prompts"][0]:
            continue

        original_prompts = [p["prompts"][0]["prompt"] for p in batch]
        user_prompt = "\n".join(original_prompts)

        response = client.messages.create(
            model="claude-3-5-sonnet-latest",
            max_tokens=1024,
            temperature=1,
            system=system_prompt,
            messages=[{"role": "user", "content": user_prompt}]
        )

        reformulated = response.content[0].text.strip()
        reformulated_list = [s.strip() for s in reformulated.split("|")]

        assert len(reformulated_list) == len(batch), "Reformulated count mismatch."

        for i, p in enumerate(batch):
            p["prompts"][0]["reformulated_prompt"] = reformulated_list[i]

        p = data / f"{prompt_type}_synthetic.json"
        p.write_text(json.dumps(prompts, indent=4))

building_location
skipping building_location


100%|██████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 28581.29it/s]


wheelchair_accessible
skipping wheelchair_accessible


100%|███████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 4473.92it/s]


open_now
skipping open_now


...

100%|█████████████████████████████████████████████████████████████| 2/2 [00:19<00:00,  9.79s/it]


opening_hours


100%|█████████████████████████████████████████████████████████████| 4/4 [00:38<00:00,  9.58s/it]


open_website


100%|█████████████████████████████████████████████████████████████| 4/4 [00:37<00:00,  9.26s/it]


In [23]:
process.keys()

dict_keys(['building_location', 'wheelchair_accessible', 'open_now', 'closed_until', 'open_until', 'navigation_link', 'opening_hours', 'open_website'])